# Web Scraping of RI Ufal - Main Campus

## 1. Importing the required libraries

In [ ]:
# Importing the required libraries.
import scrapy, csv, re, pandas as pd, numpy as np
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from scrapy.crawler import CrawlerProcess

## 2. Defining and executing the Spider class

In [ ]:
# Creating the data repository.
data = list()

In [ ]:
# Determining the URL of target pages.
urls = {
    "Computação Gráfica": "https://www.repositorio.ufal.br/browse?type=department&order=ASC&rpp=20&value=Análise%3B+Geometria+Diferencial%3B+Sistemas+dinâmicos%3B+Computação+gráfica",
    "Ciências da Computação": "https://www.repositorio.ufal.br/browse?type=department&order=ASC&rpp=20&value=Curso+de+Ciências+da+Computação",
    "Ciências da Computação - Bacharelado": "https://www.repositorio.ufal.br/browse?type=department&order=ASC&rpp=20&value=Curso+de+Ciências+da+Computação+-+Bacharelado",
    "Engenharia da Computação - Bacharelado1": "https://www.repositorio.ufal.br/browse?type=department&order=ASC&rpp=20&value=Curso+de+Engenharia+da+Computação+-+Bacharelado",
    "Engenharia da Computação - Bacharelado2": "https://www.repositorio.ufal.br/browse?type=department&order=ASC&rpp=20&value=Curso+Engenharia+da+Computação+-+Bacharelado",
    "Engenharia da Computação": "https://www.repositorio.ufal.br/browse?type=department&order=ASC&rpp=20&value=Curso+Engenharia+da+Computação",
    "Sistemas de Informação": "https://www.repositorio.ufal.br/browse?type=department&order=ASC&rpp=20&value=Curso+de+Sistema+de+Informação",
    "Sistemas de Informação - Bacharelado": "https://www.repositorio.ufal.br/browse?type=department&order=ASC&rpp=20&value=Curso+de+Sistema+de+Informação+-+Bacharelado",
    "Modelagem Computacional de Conhecimento": "https://www.repositorio.ufal.br/browse?type=department&order=ASC&rpp=20&value=Modelagem+Computacional+de+Conhecimento",
    "PPg Informática": "https://www.repositorio.ufal.br/browse?type=program&order=ASC&rpp=20&value=Programa+de+Pós-Graduação+em+Informática",
    "PPg Modelagem Computacional de Conhecimento": "https://www.repositorio.ufal.br/browse?type=program&order=ASC&rpp=20&value=Programa+de+Pós-Graduação+em+Modelagem+Computacional+de+Conhecimento"
}

In [ ]:
# Definition of Spider class.
class Spider_RI_Ufal(scrapy.Spider):
    name = "scraper_ri_ufal"

    # Start point to run the spider.
    def start_requests(self):
        for course, url in urls.items():
            yield scrapy.Request(url=url, callback=self.parse_thesis_links, meta={"course": course})

    def parse_thesis_links(self, response):
        course = response.meta["course"] if "course" in response.meta else None
        links = response.meta["links"] if "links" in response.meta else dict()
        offset = response.meta["offset"] if "offset" in response.meta else 0

        # Getting the relative URLs of the documents.
        if course not in links:
            links[course] = list()
        css = "#content > *:nth-child(2) > div:nth-child(2) > div:nth-child(4) > table > tr > td[headers='t2'] > a"
        soup = BeautifulSoup(response.text, "html.parser")
        urls_thesis = soup.select(css)
        urls_thesis = [urljoin(response.url, item["href"]) for item in urls_thesis if item.has_attr("href")]
        if len(urls_thesis) > 0:
            links[course].extend(urls_thesis)

        # Navigating among the next pages.
        css = "#content > *:nth-child(2) > div:nth-child(2) > div:nth-child(4) > div:nth-child(3)"
        url = soup.select_one(css)
        url = soup.find_all("a", string=re.compile(r"(next|próximo)", flags=re.IGNORECASE))
        if len(url) > 0:
            offset += 20
            url = f"{urls[course]}&offset={offset}"
            yield response.follow(url=url, callback=self.parse_thesis_links,
                                  meta={"course": course, "links": links, "offset": offset})
        else:
            for url in links[course]:
                yield response.follow(url=url, callback=self.parse_data,
                                      meta={"course": course})

    def parse_data(self, response):
        course = response.meta["course"] if "course" in response.meta else None
        record = {"course": course, "url": response.url}

        # Getting the HTML of page.
        soup = BeautifulSoup(response.text, "html.parser")
        css = "#content > *:nth-child(2) > div:first-child > * > a:nth-child(2)"
        record["campus"] = soup.select_one(css).text
        css = "#content > *:nth-child(2) > div:first-child > * > a:nth-child(3)"
        record["department"] = soup.select_one(css).text
        css = "#content > *:nth-child(2) > div:nth-child(2) > table:nth-child(2)"
        table = soup.select_one(css)

        # Extracting the data.
        try:
            for tag in table.select("table > tr")[:-1]:
                label = tag.select_one("tr > td.metadataFieldLabel")
                label = label.text.strip().lower().replace(":", "")
                record[label] = tag.select_one("tr > td.metadataFieldValue")
                record[label] = tuple(record[label].stripped_strings) \
                    if label == "palavras-chave" else \
                    record[label].get_text(separator=";").strip().replace("Resumo", "") \
                    if record[label] is not None else None
            css = "div.panel.panel-info > table > tr:nth-child(2) > td:first-child > a"
            record["document_url"] = soup.select_one(css)
            if record["document_url"] is not None:
                record["document_url"] = record["document_url"]["href"]
        except Exception as e:
            raise e
        data.append(record)

In [ ]:
# Execution Process to run the spider.
process = CrawlerProcess()
process.crawl(Spider_RI_Ufal)
process.start()

In [ ]:
# Checking the data.
len(data)

## 4. Preprocessing the data

In [ ]:
# Function to clean any text.
def clean_text(text):
    text = re.sub(r"\s+", " ", text, flags=re.IGNORECASE).strip()
    text = text.replace("- ", "-").replace("\ufeff", "").replace("\xad", "")
    return text

In [ ]:
# Creating the dataframe.
df = pd.DataFrame(data)

In [ ]:
# Listing the information about the dataset.
df.info()

In [ ]:
# Handling the nullable values.
df.replace({np.nan: None}, inplace=True)

In [ ]:
# Renaming the columns.
df.rename(columns={"tipo": "thesis_type", "título": "title", "autor(es)": "authors",
                   "primeiro orientador": "main_advisor", "resumo": "pt_abstract",
                   "abstract": "en_abstract", "palavras-chave": "auth_keywords",
                   "cnpq": "cnpq_area", "idioma": "language", "país": "country",
                   "sigla da instituição": "affil_accronym", "editor": "publisher",
                   "data do documento": "defense_date", "citação": "citation",
                   "título(s) alternativo(s)": "second_title",
                   "metadata.dc.contributor.advisor-co1": "second_advisor",
                   "segundo orientador": "second_advisor2",
                   "metadata.dc.contributor.referee1": "committee",
                   "metadata.dc.contributor.referee2": "committee2",
                   "metadata.dc.contributor.referee3": "committee3",
                   "metadata.dc.contributor.referee4": "committee4",
                   "metadata.dc.contributor.referee5": "committee5",
                   "metadata.dc.publisher.department": "department2",
                   "metadata.dc.publisher.program": "department3",
                   "tipo de acesso": "access_type"}, inplace=True)

In [ ]:
# List of names to fix.
names_replace = {
    "Alejandro Cesar Frery Orgambide": "Alejandro César Frery Orgambide",
    "Alessandro Fabricio Garcia": "Alessandro Fabrício Garcia",
    "Allan Medeiros de Martins": "Allan de Medeiros Martins",
    "Alvaro Alvares de Carvalho César Sobrinho": "Álvaro Alvares de Carvalho César Sobrinho",
    "Alvaro Alvares de Carvalho Cesar Sobrinho": "Álvaro Alvares de Carvalho César Sobrinho",
    "Álvaro Alvares de Carvalho Cesar Sobrinho": "Álvaro Alvares de Carvalho César Sobrinho",
    "Álvaro Álvares de Carvalho César Sobrinho": "Álvaro Alvares de Carvalho César Sobrinho",
    "André Lage Freitas": "André Lages Freitas",
    "André Luiz Lins Aquino": "André Luiz Lins de Aquino",
    "Andre Luiz Lins de Aquino": "André Luiz Lins de Aquino",
    "Andre Luiz Lins de Aquino Aquino": "André Luiz Lins de Aquino",
    "André Magno Costa de Araujo": "André Magno Costa de Araújo",
    "Antonio Alfredo Ferreira Loureiro": "Antônio Alfredo Ferreira Loureiro",
    "António Fernando de Sousa Bezerra": "Antônio Fernando de Sousa Bezerra",
    "Antonio Marcus Nogueira Lima": "Antônio Marcus Nogueira de Lima",
    "Arturo Hernandez Dominguez": "Arturo Hernández-Domínguez",
    "Arturo Hernandez-Dominguez": "Arturo Hernández-Domínguez",
    "Arturo Hernández Domí\xadnguez": "Arturo Hernández-Domínguez",
    "Arturo Hernández Domínguez": "Arturo Hernández-Domínguez",
    "Arturo Hernández-Dominguéz": "Arturo Hernández-Domínguez",
    "Arturo Hernández-Domí\xadnguez": "Arturo Hernández-Domínguez",
    "Aydano Pomponet Machado": "Aydano Pamponet Machado",
    "Baldoino Fonseca dos Santos Neto" : "Baldoíno Fonseca dos Santos Neto",
    "Carlisson Borges Tenório": "Cárlisson Borges Tenório Galdino",
    "Cecí\xadlia Mary Fischer Rubira": "Cecília Mary Fischer Rubira",
    "Cleide Jane de Sa Araujo Costa": "Cleide Jane de Sá Araujo Costa",
    "Cleide Jane de Sá Araújo Costa": "Cleide Jane de Sá Araujo Costa",
    "Credine Silva de Menezes": "Crediné Silva de Menezes",
    "Dalgoberto Miguilino Pinho Junior": "Dalgoberto Miquilino Pinho Júnior",
    "Dalgoberto Miguilino Pinho Júnior": "Dalgoberto Miquilino Pinho Júnior",
    "David Bibiano Brito": "Davi Bibiano Brito",
    "Davy de Medeiros Baia": "Davy de Medeiros Baía",
    "Denys Felipe Souza Rocha": "Denys Fellipe Souza Rocha",
    "Diego Carvalho Nascimento": "Diego Carvalho do Nascimento",
    "Diego Dermeval Medeiros da Cunha Matos": "Diego Dermeval de Medeiros da Cunha Matos",
    "Diego Dermeval Medeiros da Cunha": "Diego Dermeval de Medeiros da Cunha Matos",
    "Diego Dermeval da Cunha Matos": "Diego Dermeval de Medeiros da Cunha Matos",
    "Erick de Andrade Barbosa": "Erick de Andrade Barboza",
    "Elvys Soares Alves": "Elvys Alves Soares",
    "Fabio Jose Coutinho da Silva": "Fábio José Coutinho da Silva",
    "Fabio José Coutinho da Silva": "Fábio José Coutinho da Silva",
    "Fábio Paraguaçu": "Fábio Paraguaçu Duarte da Costa",
    "Fabio Paraguaçu Duarte da Costa": "Fábio Paraguaçu Duarte da Costa",
    "Fernando Antonio Dantas Gomes Pinto": "Fernando Antônio Dantas Gomes Pinto",
    "Giseldo da Silva Neo": "Giseldo da Silva Néo",
    "Glauber Arthur nascimento da Silva": "Glauber Arthur Nascimento da Silva",
    "Guilherme Ataí\xadde Dias": "Guilherme Ataíde Dias",
    "Heitor Judiss Savino.": "Heitor Judiss Savino",
    "Ibsen Mateus Bittencourt": "Ibsen Mateus Bittencourt Santana Pinto",
    "Icaro Bezerra Queiroz de Araújo": "Ícaro Bezerra Queiroz de Araújo",
    "Icaro Bezerra Queiroz de Araujo": "Ícaro Bezerra Queiroz de Araújo",
    "Ig Ibert Bittencourt Santanta Pinto": "Ig Ibert Bittencourt Santana Pinto",
    "Ig Ibert Bitterncourt Santana Pinto": "Ig Ibert Bittencourt Santana Pinto",
    "Ibert Bittencourt Santana Pinto": "Ig Ibert Bittencourt Santana Pinto",
    "Ig Bert Bittencourt Santana Pinto": "Ig Ibert Bittencourt Santana Pinto",
    "Ig Ibert Bittencourt": "Ig Ibert Bittencourt Santana Pinto",
    "Ig Ibert Bittencourt Santana": "Ig Ibert Bittencourt Santana Pinto",
    "Italo Carlo Lopes Silva": "Ítalo Carlo Lopes Silva",
    "Jario Jose dos Santos Junior": "Jário José dos Santos Júnior",
    "Jáiro José dos Santos Júnior": "Jário José dos Santos Júnior",
    "Jário Santos": "Jário José dos Santos Júnior",
    "Jobson de Araujo Nascimento": "Jobson de Araújo Nascimento",
    "Jonathas Patrick Hermenegildo de. Azevedo": "Jonathas Patrick Hermenegildo de Azevedo",
    "Joseana Macedo Fechine": "Joseana Macêdo Fechine Régis de Araújo",
    "Josias jordão Andrade Alves": "Josias Jordão Andrade Alves",
    "Joao Marcelo de Almeida Gusmao Lyra": "João Marcelo de Almeida Gusmão Lyra",
    "Joao Marcos Travassos Romano": "João Marcos Travassos Romano",
    "Keila Barbosa Costa": "Keila Barbosa Costa dos Santos",
    "Leandro aparecido Villas": "Leandro Aparecido Villas",
    "Leandro de Melo Sales": "Leandro Melo de Sales",
    "Leopoldo Motta Texeira": "Leopoldo Motta Teixeira",
    "Leonardo de Melo Medeiros": "Leonardo Melo de Medeiros",
    "Leonardo Pereira Viana": "Leonardo Viana Pereira",
    "Lucas Benevides Viana Amorim": "Lucas Benevides Viana de Amorim",
    "Luis Cláudius Coradine": "Luís Cláudius Coradine",
    "Luiz Cláudio Ferreira da Silva Júnior": "Luiz Claúdio Ferreira da Silva Júnior",
    "Luiz Marcos Garcia Goncalves": "Luiz Marcos Garcia Gonçalves",
    "Marcelo Costa de Oliveira": "Marcelo Costa Oliveira",
    "Manoel Álvaro de Lins Freitas Neto": "Manoel Alvaro de Freitas Lins Neto",
    "Maria Alayde Mendonçaa da Silva": "Maria Alayde Mendonça da Silva",
    "Maria Cristina Tenório C. Escarpini": "Maria Cristina Tenório Cavalcante Escarpini",
    "Maria Cristina Tenório Cabral Cavalcante": "Maria Cristina Tenório Cavalcante Escarpini",
    "Maria Cristina Tenório Cabral Cavalcante Escarpini": "Maria Cristina Tenório Cavalcante Escarpini",
    "Marí\xada Del Rosario Girardi Gutiérrez": "María Del Rosario Girardi Gutiérrez",
    "Maurí\xadcio Marengoni": "Maurício Marengoni",
    "Nuno Manuel dos Santos Antunes": "Nuno Manoel dos Santos Antunes",
    "Olival de Gusmão Freitas Junior": "Olival de Gusmão Freitas Júnior",
    "Olival de Gusmão Freitas Jr": "Olival de Gusmão Freitas Júnior",
    "Orivaldo Vieira Santana Jr": "Orivaldo Vieira de Santana Júnior",
    "Orivaldo Vieira Santana Júnior": "Orivaldo Vieira de Santana Júnior",
    "Osvaldo Anibal Rosso": "Osvaldo Aníbal Rosso",
    "Otávio José Costa de. Albuquerque Júnior": "Otávio José Costa de Albuquerque Júnior",
    "Patricia Leone Espinheira Ospina": "Patrícia Leone Espinheira Ospina",
    "Patrik Henrique da Silva Brito": "Patrick Henrique da Silva Brito",
    "Petrucio Antonio Medeiros Barros": "Petrúcio Antônio Medeiros Barros",
    "Petrucio Antônio Medeiros Barros": "Petrúcio Antônio Medeiros Barros",
    "Petrúcio Antonio Medeiros Barros": "Petrúcio Antônio Medeiros Barros",
    "Rafael Amorim da Silva": "Rafael de Amorim Silva",
    "Rafael Amorim Silva": "Rafael de Amorim Silva",
    "Ranilson Oscar Araujo Paiva": "Ranilson Oscar Araújo Paiva",
    "Renato Ambrósio Jr": "Renato Ambrósio Júnior",
    "Renato Ambrósio Jr.": "Renato Ambrósio Júnior",
    "Rodrigo de Barros": "Rodrigo de Barros Paes",
    "Romulo Nunes de Oliveira": "Rômulo Nunes de Oliveira",
    "Seiji Isotoni": "Seiji Isotani",
    "Thales Vieira": "Thales Miranda de Almeida Vieira",
    "Thales Miranda Vieira": "Thales Miranda de Almeida Vieira",
    "Thomas Lewiner": "Thomas Maurice Lewiner"
}

In [ ]:
# Normalizing the column "second_advisor".
df.loc[df.second_advisor2.notnull() & df.second_advisor.notnull(), "second_advisor"] = \
df.loc[df.second_advisor2.notnull() & df.second_advisor.notnull(),
       ["second_advisor", "second_advisor2"]].apply(
           lambda x: tuple([x.second_advisor, x.second_advisor2]), axis=1)
df.loc[df.second_advisor2.notnull() & df.second_advisor.isnull(), "second_advisor"] = \
df.loc[df.second_advisor2.notnull() & df.second_advisor.isnull(), "second_advisor2"].apply(
    lambda x: tuple([x]))
df.second_advisor = df.second_advisor.apply(
    lambda x: x if isinstance(x, tuple) else tuple([x.replace("  ", " ")]) \
        if x is not None else None)
df.second_advisor = df.second_advisor.apply(lambda row: tuple(
    [f"{x.split(",")[-1].strip()} {x.split(",")[0].strip()}" \
     if len(x.split(",")) > 1 else x.strip()
     for x in row if x is not None]) if row is not None else None)
df.second_advisor = df.second_advisor.apply(lambda row: tuple(
    [names_replace[x] if x in names_replace else x for x in row])
    if row is not None else None)

In [ ]:
# Normalizing the column "department".
df.loc[df.department2.isnull() & df.department3.notnull(), "department2"] = \
df.loc[df.department2.isnull() & df.department3.notnull(), "department3"]
df.loc[df.department.isnull() & df.department2.notnull(), "department"] = \
df.loc[df.department.isnull() & df.department2.notnull(), "department2"]
df.department.replace({
    "IC - INSTITUTO DE COMPUTAÇÃO": "Instituto de Computação",
    "IM - INSTITUTO DE MATEMÁTICA": "Instituto de Matemática"}, inplace=True)

In [ ]:
# Normalizing the column "committee".
df.loc[:, "committee"] = df.loc[:, ["committee", "committee2", "committee3",
                                    "committee4", "committee5"]].apply(
    lambda x: tuple([x.committee, x.committee2, x.committee3,
                     x.committee4, x.committee5]), axis=1)
df.committee = df.committee.apply(lambda x: tuple(
    [i.replace(":  ", "") for i in x if i is not None]))
df.committee = df.committee.apply(lambda row: tuple(
    [f"{x.split(",")[-1].strip()} {x.split(",")[0].strip()}" \
     if len(x.split(",")) > 1 else x.strip()
     for x in row if x is not None]) if row is not None else None)
df.committee = df.committee.apply(lambda row: tuple(
    [names_replace[x] if x in names_replace else x for x in row])
    if row is not None else None)

In [ ]:
# Normalizing the features "course", "campus", and "language".
df.course.replace({
    "Engenharia da Computação - Bacharelado2": "Engenharia da Computação",
    "Engenharia da Computação - Bacharelado1": "Engenharia da Computação",
    "Sistemas de Informação - Bacharelado": "Sistemas de Informação",
    "Ciências da Computação - Bacharelado": "Ciências da Computação"},
    inplace=True)
df.campus.replace("00 CAMPUS ARISTÓTELES CALAZANS SIMÕES (CAMPUS A. C. SIMÕES)",
                  "Campus A.C. Simões", inplace=True)
df.language.replace("português", "por", inplace=True)

In [ ]:
# Normalizing the features "authors", and "main_advisor".
df.loc[:, ["authors", "main_advisor"]] = df.loc[:,
    ["authors", "main_advisor"]].apply(lambda row: row.apply(
        lambda x: re.sub(r"\([^\(\)]+\)", "",
        f"{x.split(",")[-1].strip()} {x.split(",")[0].strip()}" \
        if len(x.split(",")) > 1 else x).replace("  ", " ")))
df.loc[:, ["authors", "main_advisor"]] = df.loc[:,
    ["authors", "main_advisor"]].apply(lambda row: row.apply(
    lambda x: tuple([names_replace[x] \
        if x in names_replace else x.strip()]) \
            if x is not None else None))

In [ ]:
# Normalizing the feature "cnpq_area".
df.cnpq_area.replace({
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO::METODOLOGIA E TECNICAS DA COMPUTACAO::ENGENHARIA DE SOFTWARE": "Engenharia de Software",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO": "Ciência da Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO::SISTEMAS DE COMPUTACAO": "Sistemas de Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO::METODOLOGIA E TECNICAS DA COMPUTACAO::SISTEMAS DE INFORMACAO": "Sistemas de Informação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::MATEMATICA": "Matemática",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO::TEORIA DA COMPUTACAO::COMPUTABILIDADE E MODELOS DE COMPUTACAO": "Teoria da Computação",
    "CNPQ::ENGENHARIAS;CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO": "Ciência da Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO::SISTEMAS DE COMPUTACAO::ARQUITETURA DE SISTEMAS DE COMPUTACAO": "Sistemas de Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO;CNPQ::ENGENHARIAS": "Ciência da Computação",
    "CNPQ::CIENCIAS SOCIAIS APLICADAS::CIENCIA DA INFORMACAO": "Ciência da Informação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO::MATEMATICA DA COMPUTACAO": "Matemática da Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::MATEMATICA::MATEMATICA APLICADA": "Matemática",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::MATEMATICA::GEOMETRIA E TOPOLOGIA::SISTEMAS DINAMICOS": "Sistemas Dinâmicos",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO;CNPQ::CIENCIAS HUMANAS::EDUCACAO": "Ciência da Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA": "Ciência da Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO;CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO::SISTEMAS DE COMPUTACAO": "Sistemas de Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO;Recommender Systems": "Ciência da Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::PROBABILIDADE E ESTATISTICA": "Matemática",
    "CNPQ::CIENCIAS DA SAUDE::MEDICINA;CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO": "Ciência da Computação",
    "CNPQ::CIENCIAS DA SAUDE::FISIOTERAPIA E TERAPIA OCUPACIONAL": "Fisioterapia",
    "CIENCIA DA COMPUTACAO": "Ciência da Computação",
    "CNPQ::CIENCIAS EXATAS E DA TERRA::CIENCIA DA COMPUTACAO;CNPQ::CIENCIAS DA SAUDE::MEDICINA": "Ciência da Computação"}, inplace=True)

In [ ]:
# Normalizing the feature "defense_date".
months = {"fev": "feb", "abr": "apr", "mai": "may",
          "ago": "aug", "set": "sep", "out": "oct",
          "dez": "dec"}
df.defense_date.replace({"2008": "1-dez-2008", "nov-2009": "1-nov-2009",
                         "dez-2006": "1-dez-2006", "2020": "1-jan-2020",
                         "14-jul-  22": "14-jul-2022"}, inplace=True)
df.defense_date = df.defense_date.apply(
    lambda x: f"{x.split("-")[-1]}-{months[x.split("-")[1]]}-{x.split("-")[0]}" \
        if x.split("-")[1] in months else \
        f"{x.split("-")[-1]}-{x.split("-")[1]}-{x.split("-")[0]}")
df.defense_date = pd.to_datetime(df.defense_date, format="%Y-%b-%d")

In [ ]:
# Normalizing the feature "document_url".
df.document_url = df.loc[:, ["url", "document_url"]].apply(
    lambda x: urljoin(x.url, x.document_url), axis=1)

In [ ]:
# Removing the unecessary columns.
df.drop(columns=["department2", "department3", "second_advisor2",
                 "committee2", "committee3", "committee4",
                 "committee5", "country", "citation"], inplace=True)

In [ ]:
# Normalizing the column "auth_keywords".
df.loc[df.auth_keywords.notnull(), "auth_keywords"] = df.loc[
    df.auth_keywords.notnull(), "auth_keywords"].apply(lambda x: tuple(
        [clean_text(k).strip() for k in x
            if len(clean_text(k).strip()) > 1]))

In [ ]:
# Normalizing the columns "title", "second_title", "pt_abstract", and "en_abstract".
df.loc[df.title.notnull(), "title"] = df.loc[
    df.title.notnull(), "title"].apply(clean_text)
df.loc[df.second_title.notnull(), "second_title"] = df.loc[
    df.second_title.notnull(), "second_title"].apply(clean_text)
df.loc[df.pt_abstract.notnull(), "pt_abstract"] = df.loc[
    df.pt_abstract.notnull(), "pt_abstract"].apply(clean_text)
df.loc[df.en_abstract.notnull(), "en_abstract"] = df.loc[
    df.en_abstract.notnull(), "en_abstract"].apply(clean_text)

In [ ]:
# Checking the result.
df.head()

## 4. Saving the data

In [ ]:
# Saving the data into a CSV file.
df.to_csv("../data/ufal_main_campus_thesis.csv", index=False, quoting=csv.QUOTE_ALL)